# Diurnal variations of wind 


This notebook look into the wind velocity parameters from the model outputs 

In [1]:
import xarray as xr
from pathlib import Path
import pandas as pd
import intake
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import easygems.healpix as egh

In [2]:
data_2d_z2 = "/g/data/qx55/uk_node/glm.n2560_RAL3p3/data.healpix.PT1H.z6.zarr" #PT1H is hourly data
ds = xr.open_zarr(data_2d_z2)
ds

<xarray.Dataset> Size: 49GB
Dimensions:  (cell: 49152, time: 10489)
Coordinates:
  * cell     (cell) int64 393kB 0 1 2 3 4 5 ... 49147 49148 49149 49150 49151
    crs      float64 8B ...
  * time     (time) datetime64[ns] 84kB 2020-01-20 ... 2021-04-01
Data variables: (12/24)
    clivi    (time, cell) float32 2GB dask.array<chunksize=(64, 16384), meta=np.ndarray>
    clt      (time, cell) float32 2GB dask.array<chunksize=(64, 16384), meta=np.ndarray>
    clwvi    (time, cell) float32 2GB dask.array<chunksize=(64, 16384), meta=np.ndarray>
    hflsd    (time, cell) float32 2GB dask.array<chunksize=(64, 16384), meta=np.ndarray>
    hfssd    (time, cell) float32 2GB dask.array<chunksize=(64, 16384), meta=np.ndarray>
    huss     (time, cell) float32 2GB dask.array<chunksize=(64, 16384), meta=np.ndarray>
    ...       ...
    rsut     (time, cell) float32 2GB dask.array<chunksize=(64, 16384), meta=np.ndarray>
    rsutcs   (time, cell) float32 2GB dask.array<chunksize=(64, 16384), meta=np.ndarray>
    tas      (time, cell) float32 2GB dask.array<chunksize=(64, 16384), meta=np.ndarray>
    ts       (time, cell) float32 2GB dask.array<chunksize=(64, 16384), meta=np.ndarray>
    uas      (time, cell) float32 2GB dask.array<chunksize=(64, 16384), meta=np.ndarray>
    vas      (time, cell) float32 2GB dask.array<chunksize=(64, 16384), meta=np.ndarray>
Attributes:
    Met Office DYAMOND3 simulations:  A group of experiments have been conduc...
    bounds:                           {'lower_left_lat': -90, 'lower_left_lon...
    latitiude_convention:             [-90, 90]
    longitude_convention:             [0, 360]
    regional:                         False
    simulation:                       glm.n2560_RAL3p3
    simulation_description:           The MetUM uses a regular lat-lon grid, ...

In [7]:
# slect the wind data u and v components
u=ds.uas.sel(time=slice("2020-01-01", "2021-01-01"))
v=ds.vas.sel(time=slice("2020-01-01", "2021-01-01"))
velocity = (u**2 + v**2)**0.5

In [10]:
u

<xarray.DataArray 'uas' (time: 8352, cell: 49152)> Size: 2GB
dask.array<getitem, shape=(8352, 49152), dtype=float32, chunksize=(64, 16384), chunktype=numpy.ndarray>
Coordinates:
  * cell     (cell) int64 393kB 0 1 2 3 4 5 ... 49147 49148 49149 49150 49151
    crs      float64 8B ...
  * time     (time) datetime64[ns] 67kB 2020-01-20 ... 2021-01-01T23:00:00
Attributes:
    STASH:          [1, 3, 225]
    UM_name:        x_wind
    grid_mapping:   healpix_nested
    healpix_zoom:   6
    long_name:      eastward_wind
    source:         Data from Met Office Unified Model
    standard_name:  x_wind
    um_version:     13.5
    units:          m s-1

In [27]:
# funtion to get the average velocity for months. output is 24 time steps for each month available
def monthly_hourly_average(data):
    """
    Compute the time average for each hour of the day across all days, separately for each month.
    """
    # Group by both month and hour in a single step
    monthly_hourly_avg = data.groupby(['time.month', 'time.hour']).mean('time')
    return monthly_hourly_avg


In [37]:
# get the average velocities 
hourly_avg_u = diurnal_cycle_average(u)  
hourly_avg_v = monthly_hourly_average(v)
hourly_avg_velocity = monthly_hourly_average(velocity)
hourly_avg_velocity

AttributeError: 'DataArrayGroupBy' object has no attribute 'groupby'

In [ ]:
# plot the average wind components of a single cell for averages of a month 
plt.plot(hourly_avg_u.isel(cell=5,month=1),hourly_avg_v.isel(cell=5,month=1))

In [ ]:
# plot the average values for a singles cell.
hourly_avg_u.isel(cell=50).plot() 

In [ ]:
# Plotting lat/lon values over the grid
projection=ccrs.PlateCarree(central_longitude=0.0)

ax = egh.create_geoaxis(add_coastlines=True,projection=projection)
im = egh.healpix_show(hourly_avg_velocity.isel(month=0,hour=0), ax=ax, add_coastlines=True)
ax.set_title(f'hourly average wind velocity')
ax.gridlines(draw_labels=True)
fig=ax.get_figure()
fig.colorbar(im,orientation='horizontal')

In [ ]:
# Plotting the above, over Australia

# Note - we still have to use the egh wrapper functions to plot the healpix gridded data
# onto a lat/lon grid
projection = ccrs.PlateCarree(central_longitude=0)

ax = egh.create_geoaxis(add_coastlines=True,projection=projection)

ax.set_extent([105, 160,-50, -5], crs=ccrs.PlateCarree())
im = egh.healpix_show(hourly_avg_velocity.isel(month=0,hour=0), ax=ax)
#ax.coastlines()
ax.set_title(f'hourly wind healpix zoom 5')
ax.gridlines(draw_labels=True)
fig=ax.get_figure()
fig.colorbar(im,orientation='horizontal')

In [ ]:
monthly_max=hourly_avg_velocity.max(dim='hour') #looking at max values for each months 
#monthly_max.isel(cell=0,month=1).plot()
monthly_max

In [ ]:
monthly_max.isel(cell=7).plot()

In [ ]:
avg_max = hourly_avg_velocity.max(dim='hour').max(dim='month')
avg_min = hourly_avg_velocity.min(dim='hour').min(dim='month') 
avg_min
 

In [ ]:
avg_max.isel(cell=0).values

In [ ]:
max_min_diff = avg_max - avg_min
max_min_diff

In [ ]:
projection=ccrs.PlateCarree(central_longitude=0.0)
ax = egh.create_geoaxis(add_coastlines=True,projection=projection)
im = egh.healpix_show(max_min_diff, ax=ax, add_coastlines=True)
ax.set_title(f'yearly max_min difference velocity')
ax.gridlines(draw_labels=True)
fig=ax.get_figure()
fig.colorbar(im,orientation='horizontal')

### picking the timestamp for max valocity

In [3]:

#  Reshape to stack (month, hour) into a single dimension for easier reduction
uas_stacked = hourly_avg_velocity.stack(time_combo=("month", "hour"))
uas_stacked

NameError: name 'hourly_avg_velocity' is not defined

In [4]:
# Find the index of the max along the combined dimension, for each cell
max_idx = uas_stacked.argmax(dim="time_combo")
max_idx.values

NameError: name 'uas_stacked' is not defined

In [5]:
# Step 3: Use that index to get the actual month and hour
months = hourly_avg_velocity.coords['month'].values
hours = hourly_avg_velocity.coords['hour'].values
months 

NameError: name 'hourly_avg_velocity' is not defined

In [6]:
month_indices, hour_indices = np.unravel_index(max_idx.values, (12, 24))

NameError: name 'np' is not defined

In [7]:
# Step 4: Also get the max value for each cell
max_val = uas_stacked.max(dim="time_combo")

NameError: name 'uas_stacked' is not defined

In [8]:
max_values_ds = xr.Dataset({
    "max_value": (["cell"], max_val.values),
    "month_of_max": (["cell"], month_indices),
    "hour_of_max": (["cell"], hour_indices),
})

NameError: name 'max_val' is not defined

In [9]:
max_values_ds

NameError: name 'max_values_ds' is not defined

In [ ]:
projection=ccrs.PlateCarree(central_longitude=0.0)
ax = egh.create_geoaxis(add_coastlines=True,projection=projection)
im = egh.healpix_show(max_values_ds.hour_of_max, ax=ax, add_coastlines=True)
ax.set_title(f'Hour of maximum wind velocity')
ax.gridlines(draw_labels=True)
fig=ax.get_figure()
fig.colorbar(im,orientation='horizontal')

In [10]:
from matplotlib.colors import LinearSegmentedColormap

# Create custom colormap: light → dark → light
gray_fade = LinearSegmentedColormap.from_list(
    "gray_fade", ["lightgray", "black", "lightgray"]
)

In [11]:
projection=ccrs.PlateCarree(central_longitude=0.0)
ax = egh.create_geoaxis(add_coastlines=True,projection=projection)
im = egh.healpix_show(max_values_ds.hour_of_max, ax=ax, add_coastlines=True,cmap=gray_fade)
ax.set_title(f'Hour of maximum wind velocity')
ax.gridlines(draw_labels=True)
fig=ax.get_figure()
fig.colorbar(im,orientation='horizontal')

NameError: name 'ccrs' is not defined